In [1]:
# Scientific libraries
import numpy as np
#import scipy

# import Pandas

import pandas as pd
import astropy.io.fits as fits
import astropy.units as u
import astropy.constants as const

# Graphic libraries

import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook
from jupyterthemes import jtplot
jtplot.style('oceans16', context='notebook', fscale=1)


#optional 3ML imports

from threeML import *
#from astromodels.xspec.xspec_settings import *
#from astromodels.xspec.factory import *

#useful libraries

from glob import glob
import copy
import collections
#import warnings
#warnings.simplefilter('ignore')

Configuration read from /Users/jburgess/.threeML/threeML_config.yml


In [2]:
gbm_cat = FermiGBMBurstCatalog()

Building cache for fermigbrst.



In [4]:
gbm_cat.query_sources('GRB080916009')

name,ra,dec,trigger_time,t90
object,float64,float64,float64,float64
GRB080916009,119.800,-56.600,54725.0088613,62.977


In [7]:
model = gbm_cat.get_model()

In [10]:
m = model['GRB080916009']

In [11]:
m.GRB080916009.spectrum.main(10)

array(0.16619294764912226)

In [13]:
import astropy.units as u

In [14]:
a=10*u.keV

In [17]:
type(a)

astropy.units.quantity.Quantity

In [18]:
isinstance(a,u.Quantity )

True

In [20]:
import scipy.integrate as integrate

In [30]:
class HRCalc(object):
    
    def __init__(self, model, ene_lo_min = 50.,ene_lo_max = 300.,ene_hi_min= 300., ene_hi_max=1000.):
        """
        
        
        """
        
        
        # convert everything to keV
        
        self._ene_lo_min, warn = HRCalc._convert_to_unit(ene_lo_min)
        
        if warn:
            
            RuntimeWarning('Assuming ene_lo_min is in keV')
            
        self._ene_lo_max, warn = HRCalc._convert_to_unit(ene_lo_max)
        
        if warn:
            
            RuntimeWarning('Assuming ene_lo_max is in keV')
            
        self._ene_hi_min, warn = HRCalc._convert_to_unit(ene_hi_min)
        
        if warn:
            
            RuntimeWarning('Assuming ene_hi_min is in keV')
            
        self._ene_hi_max, warn = HRCalc._convert_to_unit(ene_hi_max)
        
        if warn:
            
            RuntimeWarning('Assuming ene_hi_max is in keV')
         
        
        # check we did not fuck up the energies
        
        assert ene_lo_min < ene_lo_max, 'energies are out of order!'
        assert ene_lo_max <= ene_hi_min, 'energies are out of order!'
        assert ene_hi_min < ene_hi_max, 'energies are out of order!'
        
        # save the model call
        
        self._call = lambda ene: model.get_point_source_fluxes(0,ene) # phts/s/cm2/keV
        self._ene_call = lambda ene: model.get_point_source_fluxes(0,ene)*ene # phts/s/cm2/keV
        
        #self._kev2erg = (1*u.keV).to(u.erg).value
        
        self._compute_photon_hardness_ratio()
        self._compute_energy_hardness_ratio()
        
    
    def _compute_photon_hardness_ratio(self):
        
        self._ph_hr_lo = integrate.quad(self._call, self._ene_lo_min, self._ene_lo_max)[0]
        self._ph_hr_hi = integrate.quad(self._call, self._ene_hi_min, self._ene_hi_max)[0]
        self._ph_hr = self._ph_hr_lo/self._ph_hr_hi
        
        
    def _compute_energy_hardness_ratio(self):
        
        self._ene_hr_lo = integrate.quad(self._ene_call, self._ene_lo_min, self._ene_lo_max)[0]
        self._ene_hr_hi = integrate.quad(self._ene_call, self._ene_hi_min, self._ene_hi_max)[0]
        self._ene_hr = self._ene_hr_lo/self._ene_hr_hi
        
        
        
    @property
    def photon_hardness_ratio(self):
        """
        :return : photon hardness ratio 
        """
        
        return self._ph_hr #* self._kev2erg 
    
    @property
    def energy_hardness_ratio(self):
        """
        :return : energy hardness ratio 
        """
        
        return self._ene_hr
        
        
    @staticmethod
    def _convert_to_unit(value):
        
        if isinstance(value,u.Quantity):
            
            value = value.to(u.keV).value
            warn = False
        else:
            
            warn = True
            
        return value, warn
    
        

In [27]:
hr = HRCalc(m)

In [28]:
hr.energy_hardness_ratio

0.7409494896473245

In [29]:
hr.photon_hardness_ratio

4.794446208244634e-09